This notebook loads the data, loads the fitted models from disk and evaluates the models.

## Loading Data

## Structure Perceptron

### Structure Perceptron w/ given features

### Structure Perceptron w/ extra features

## Deep Learning Approach